<table class="notebook-buttons" align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/JeremieGince/DefiProgFest2023/blob/main/Gym-Tutorial.ipynb"><img src="https://github.com/NeuroTorch/NeuroTorch/blob/main/images/colab_logo_32px.png?raw=true" width=32px height=32px  />Run in Google Colab</a>
</td>
  <td>
    <a target="_blank" href="https://github.com/JeremieGince/DefiProgFest2023/blob/main/Gym-Tutorial.ipynb"><img src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png" width=32px height=32px />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/JeremieGince/DefiProgFest2023/blob/main/Gym-Tutorial.ipynb"><img src="https://github.com/NeuroTorch/NeuroTorch/blob/main/images/download_logo_32px.png?raw=true" width=32px height=32px />Download notebook</a>
  </td>
</table>

In [2]:
import gym
import numpy as np

# Gym - Lunar lander

Liens vers la documentation: 
- [Gym-basic_usage](https://www.gymlibrary.dev/content/basic_usage/)
- [Gym-core](https://www.gymlibrary.dev/api/core/)
- [Lunar Lander](https://www.gymlibrary.dev/environments/box2d/lunar_lander/)

Cet environnement `Gym` est un problème d'optimisation qui nécessite de faire atterrir (alunir?) un *lunar lander* entre deux drapeaux en lui envoyant des commandes en temps réel. Pour un environnement exécuté en mode discret (`continuous = False`), les commandes sont de forme binaire, utilisant toujours la pleine puissance du moteur. Les quatre commandes possibles sont:

- Ne rien faire (`0`)
- Activer le moteur droite (`1`)
- Activer le moteur central (`2`)
- Activer le moteur gauche (`3`)

Afin de déterminer les commandes appropriées selon l'état de la situation, les différentes coordonnées spatiales du vaisseau sont rendues accessibles dans un vecteur d'observation (`observation`) qui contient, dans l'ordre, les huit coordonnées suivantes:

- Position horizontale (intervalle `[-1.5, 1.5]`)
- Position verticale (intervalle `[1.5, -1.5]`)
- Vitesse horizontale (intervalle `[-5, 5]`)
- Vitesse verticale (intervalle `[-5, 5]`)
- Angle (intervalle  `[-3.14, 3.14]`)
- Vitesse angulaire (intervalle `[-5, 5]`)
- Contact pied gauche (`True` or `False`)
- Contact pied droite (`True` or `False`)

L'environnement est initialisé aléatoirement, puis exécuté dans une boucle. À chaque itération de la boucle, les coordonnées du vaisseau sont mises à jour, puis un ensemble de variables est retourné à l'utilisateur, qui doit utiliser celles-ci afin de déterminer la prochaine commande.

Jetons un coup d'oeil à une procédure d'atterrissage où les commandes sont déterminées aléatoirement à chaque itération:

In [2]:
env = gym.make("LunarLander-v2", render_mode="human")

observation, info = env.reset(seed=None)
terminal = False
while not terminal:
    action = env.action_space.sample()
    
    observation, reward, done, truncated, info = env.step(action)
    terminal = done or truncated
    
env.close()

E:\Github\DefiProgFest2023\venv\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Sans grande surprise, l'atterrissage est une véritable catastrophe.

À chaque itération, une valeur de récompense (`reward`) est retournée à l'utilisateur. Des valeurs positives sont attribuées lorsque le vaisseau se dirige vers les drapeaux, ainsi que lorsque l'atterrissage est réussi. À l'inverse, des valeurs négative sont retournées à chaque fois que les moteurs sont utilisés, ainsi que lorsque le vaisseau collisionne avec le sol (pour plus de détails sur le système de pointages, voir la documentation).

Nous pouvons ainsi cumuler les récompenses sur toute la durée de la simulation, afin d'obtenir un score de récompense totale qui permet de juger de la performance de la procédure d'atterrissage. Effectuons l'évaluation de notre pilote aléatoire:

In [3]:
env = gym.make("LunarLander-v2", render_mode="human")

observation, info = env.reset(seed=None)

terminal = False
cumulative_rewards = 0

while not terminal:
    action = env.action_space.sample()
    
    observation, reward, done, truncated, info = env.step(action)
    cumulative_rewards += reward
    terminal = done or truncated

print(f'Cumulative Rewards: {cumulative_rewards:.3f}')
if cumulative_rewards < 0:
    print(f"The eagle has not landed.")
if cumulative_rewards >= 200:
    print(f"The eagle has landed.")
env.close()

Cumulative Rewards: -103.558
The eagle has not landed.


De toute évidence, il serait possible de faire mieux. Il faut désormais exploiter le vecteur `observation` retourné à chaque itération, soit

In [4]:
print(observation)

[-0.3570746  -0.08372779 -0.74975836 -0.709012   -0.04089577  4.172947
  1.          0.        ]


Ce dernier devrait contenir toute l'information nécessaire afin d'établir une procédure qui permet de déterminer, à chaque itération, la bonne commande à effectuer.

Vous devez maintenant concevoir un algorithme qui permet de faire atterrir le vaisseau **tout en cumulant le plus de points possible**. Compte tenu la nature aléatoire de la tâche, la performance moyenne sur 100 simulations sera évaluée. Pour accélérer l'exécution des simulations, le mode `render_mode=rgb_array`. Bien entendu, vous devriez utiliser le mode `human` pour visualiser les performances de votre algorithme pendant son développement.

In [5]:
cumulative_rewards_list = []

env = gym.make("LunarLander-v2", render_mode="rgb_array")
for _ in range(100):
    observation, info = env.reset(seed=None)
    
    terminal = False
    cumulative_rewards = 0
    
    while not terminal:
        action = env.action_space.sample()
        observation, reward, done, truncated, info = env.step(action)
        cumulative_rewards += reward
        terminal = done or truncated

    cumulative_rewards_list.append(cumulative_rewards)

env.close()

In [6]:
print(f"Average cumulative rewards: {np.mean(cumulative_rewards_list):.3f}")
if np.mean(cumulative_rewards_list) < 0:
    print("The eagle has, on average, not landed.")
if np.mean(cumulative_rewards_list) >= 200:
    print(f"The eagle has, on average, landed.")

Average cumulative rewards: -171.932
The eagle has, on average, not landed.


La balle est maintenant dans votre camp. The eagle *must* land.

# Comment tester son agent

In [8]:
from tools.tester import LunarLanderPerformanceTest, LunarLanderTester, LunarLanderPEP8Test
from main import get_env_configs
from typing import Union

### L'agent aléatoire

Ici, on créé notre agent pour éventuellement le tester. Cette class vient du fichier [lunar_lander_agent.py](./lunar_lander_agent.py) où vous allez devoir implémenter le votre.

In [9]:
class LunarLanderAgent:
	def __init__(self, env_config: dict, **kwargs):
		self.env_config = env_config
		self.set_default_env_config()
		self.observation_as_rgb = self.env_config.get("render_mode", None) == "rgb_array"
	
	def set_default_env_config(self):
		self.env_config.setdefault('render_mode', None)
		self.env_config.setdefault('id', "LunarLander-v2")
	
	def make_env(self):
		env = gym.make(**self.env_config)
		return env
	
	def get_action(self, observation: np.ndarray) -> Union[int, np.ndarray]:
		env = self.make_env()
		action = env.action_space.sample()
		env.close()
		return action

### Les tests

Dans la prochaine cellule, on vous montre le code à utiliser pour tester son agent dans l'environnement de test.

In [10]:
configs_file_path = "./env_configs.json"

# On instantie l'objet de test
tester = LunarLanderTester()

# On ajoute le test de PEP8
# file_path doit être égal à votre fichier `lunar_lander_agent.py` où vous allez implémenter votre agent.
pep8_test = LunarLanderPEP8Test(name="PEP8", file_path="./lunar_lander_agent.py")
tester.add_test(pep8_test)

# On ajoute les tests de performances en utilisant les configurations de test.
env_configs = get_env_configs(configs_file_path=configs_file_path)
for config_name, env_config in env_configs.items():
    performance_test = LunarLanderPerformanceTest(
        name=config_name,
        agent=LunarLanderAgent(env_config=env_config),
        env_config=env_config,
    )
    tester.add_test(performance_test)

# On lance les tests
tester.run()

# On affiche les tests
print(tester)

# On sauvegarde les résultats 
tester.to_file(file_path="./test_results.txt")

Running Echelon 4: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:32<00:00,  3.12it/s, [Echelon 4: 0.00 %]]

[PEP8: 100.00 %]
[Echelon 0: 0.00 %]
[Echelon 1: 0.00 %]
[Echelon 2: 0.00 %]
[Echelon 3: 0.00 %]
[Echelon 4: 0.00 %]
Test results saved to './test_results.txt'.
